In [1]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 5.3 MB/s 
     |████████████████████████████████| 1.6 MB 49.8 MB/s 


In [2]:
import kerastuner as kt

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [3]:
import numpy as np
import pandas as pd

In [4]:
data = pd.read_csv("diabetes.csv")

In [5]:
data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [6]:
data.corr()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
Pregnancies,1.000000,0.129459,0.141282,-0.081672,-0.073535,0.017683,-0.033523,0.544341,0.221898
Glucose,0.129459,1.000000,0.152590,0.057328,0.331357,0.221071,0.137337,0.263514,0.466581
BloodPressure,0.141282,0.152590,1.000000,0.207371,0.088933,0.281805,0.041265,0.239528,0.065068
SkinThickness,-0.081672,0.057328,0.207371,1.000000,0.436783,0.392573,0.183928,-0.113970,0.074752
Insulin,-0.073535,0.331357,0.088933,0.436783,1.000000,0.197859,0.185071,-0.042163,0.130548
BMI,0.017683,0.221071,0.281805,0.392573,0.197859,1.000000,0.140647,0.036242,0.292695
DiabetesPedigreeFunction,-0.033523,0.137337,0.041265,0.183928,0.185071,0.140647,1.000000,0.033561,0.173844
Age,0.544341,0.263514,0.239528,-0.113970,-0.042163,0.036242,0.033561,1.000000,0.238356
Outcome,0.221898,0.466581,0.065068,0.074752,0.130548,0.292695,0.173844,0.238356,1.000000


In [7]:
X = data.drop('Outcome', axis=1)

In [8]:
y = data['Outcome']

In [9]:
X

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33
...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63
764,2,122,70,27,0,36.8,0.340,27
765,5,121,72,23,112,26.2,0.245,30
766,1,126,60,0,0,30.1,0.349,47


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=48)

In [11]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)


In [12]:
X_test = scaler.transform(X_test)

In [13]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam, RMSprop, Adagrad, SGD

In [14]:
def build_model(hp):
  model = Sequential()

  model.add(Dense(8,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  optimizer = hp.Choice('optimizer', values = ['RMSprop', 'Adam', 'Adagrad', 'SGD'])

  model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

  return model

In [15]:
# Tuner object for KT class
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5)

In [16]:
tuner.search(X_train, y_train, epochs=10,validation_data=(X_test, y_test))

Trial 4 Complete [00h 00m 02s]
val_accuracy: 0.7857142686843872

Best val_accuracy So Far: 0.7857142686843872
Total elapsed time: 00h 00m 08s


In [17]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'RMSprop'}

In [18]:
best_model = tuner.get_best_models(num_models=1)[0]

In [19]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 72        
                                                                 
 dense_1 (Dense)             (None, 1)                 9         
                                                                 
Total params: 81
Trainable params: 81
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Will fit the best model now from the Initial_epoch as 11 as we already ran 10 previosly.
best_model.fit(X_train, y_train, epochs=100, initial_epoch=11,validation_data=(X_test, y_test))

Epoch 12/100
20/20 [==============================] - 1s 11ms/step - loss: 0.5393 - accuracy: 0.7622 - val_loss: 0.5179 - val_accuracy: 0.7727
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5293 - accuracy: 0.7541 - val_loss: 0.5091 - val_accuracy: 0.7662
Epoch 14/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5201 - accuracy: 0.7557 - val_loss: 0.5012 - val_accuracy: 0.7597
Epoch 15/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5122 - accuracy: 0.7590 - val_loss: 0.4941 - val_accuracy: 0.7662
Epoch 16/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5052 - accuracy: 0.7606 - val_loss: 0.4881 - val_accuracy: 0.7597
Epoch 17/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4992 - accuracy: 0.7638 - val_loss: 0.4834 - val_accuracy: 0.7727
Epoch 18/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4940 - accuracy: 0.7671 - val_loss: 0.4800 - val_accuracy: 0.7792
Epoch

In [21]:
# Selecting No. of Nodes in a layer
def build_model(hp):
  model = Sequential()

  units = hp.Int('units', min_value=4,max_value=96,step=4)

  model.add(Dense(units=units, activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='Adam',metrics=['accuracy'], loss='binary_crossentropy')

  return model

In [22]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5,
                        project_name = 'keras-tuner'
                        )

In [23]:
tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.7922077775001526

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 11s


In [24]:
tuner.get_best_hyperparameters()[0].values

{'units': 36}

In [25]:
best_model = tuner.get_best_models(num_models=1)[0]

In [26]:
best_model.fit(X_train, y_train, epochs=100, initial_epoch=11, validation_data=(X_test, y_test))

Epoch 12/100
20/20 [==============================] - 1s 20ms/step - loss: 0.4884 - accuracy: 0.7736 - val_loss: 0.4816 - val_accuracy: 0.8052
Epoch 13/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4810 - accuracy: 0.7736 - val_loss: 0.4775 - val_accuracy: 0.8052
Epoch 14/100
20/20 [==============================] - 0s 8ms/step - loss: 0.4761 - accuracy: 0.7736 - val_loss: 0.4736 - val_accuracy: 0.7987
Epoch 15/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4718 - accuracy: 0.7850 - val_loss: 0.4711 - val_accuracy: 0.7857
Epoch 16/100
20/20 [==============================] - 0s 10ms/step - loss: 0.4682 - accuracy: 0.7883 - val_loss: 0.4689 - val_accuracy: 0.8117
Epoch 17/100
20/20 [==============================] - 0s 10ms/step - loss: 0.4653 - accuracy: 0.7834 - val_loss: 0.4678 - val_accuracy: 0.8117
Epoch 18/100
20/20 [==============================] - 0s 9ms/step - loss: 0.4627 - accuracy: 0.7866 - val_loss: 0.4662 - val_accuracy: 0.8117
Epo

In [27]:
#For selecting no of Layers, no. of Nodes, Optimizes and Activation function
def build_model(hp):

   model = Sequential()

   count =0

   for i in range(hp.Int('num_layers', min_value=1, max_value=10)):

     if count == 0:
       model.add(Dense(hp.Int('units' + str(i), min_value=8, max_value=128, step=8),
                       activation = hp.Choice('activation' + str(i), values=['sigmoid','relu','tanh','elu']),
                       input_dim=8
                       )
       )
     else:

       model.add(Dense(hp.Int('units' + str(i), min_value=8, max_value=128, step=8),
                       activation = hp.Choice('activation' + str(i), values=['sigmoid','relu','tanh','elu']),
                       )
       )
       count+=1

   model.add(Dense(1, activation='sigmoid'))

   model.compile(optimizer=hp.Choice('optimizer', values=['rmsprop','adam','adagrad','adadelta','sgd']),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
   return model


In [28]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=8,
                        directory = 'Tuner',
                        project_name='Final_Tuning')

In [29]:
tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Trial 8 Complete [00h 00m 02s]
val_accuracy: 0.7077922224998474

Best val_accuracy So Far: 0.7922077775001526
Total elapsed time: 00h 00m 20s


In [30]:
# using 10 epochs
tuner.get_best_hyperparameters()[0].values

{'num_layers': 1, 'units0': 120, 'activation0': 'elu', 'optimizer': 'adam'}

In [31]:
final_model = tuner.get_best_models(num_models=1)[0]

In [32]:
final_model.fit(X_train, y_train, epochs=100, initial_epoch=11, validation_data=(X_test, y_test))

Epoch 12/100
20/20 [==============================] - 1s 10ms/step - loss: 0.4884 - accuracy: 0.7622 - val_loss: 0.4776 - val_accuracy: 0.7857
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4791 - accuracy: 0.7638 - val_loss: 0.4739 - val_accuracy: 0.7857
Epoch 14/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4756 - accuracy: 0.7655 - val_loss: 0.4707 - val_accuracy: 0.7857
Epoch 15/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4730 - accuracy: 0.7687 - val_loss: 0.4712 - val_accuracy: 0.7922
Epoch 16/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4725 - accuracy: 0.7655 - val_loss: 0.4729 - val_accuracy: 0.7857
Epoch 17/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4721 - accuracy: 0.7671 - val_loss: 0.4715 - val_accuracy: 0.7857
Epoch 18/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4712 - accuracy: 0.7655 - val_loss: 0.4750 - val_accuracy: 0.7857
Epoch

In [44]:
print(tuner.get_best_hyperparameters()[0].values)

{'num_layers': 1, 'units0': 120, 'activation0': 'elu', 'optimizer': 'adam'}


In [45]:
tuner.search_space_summary()

Search space summary
Default search space size: 20
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 10, 'step': 1, 'sampling': None}
units0 (Int)
{'default': None, 'conditions': [], 'min_value': 8, 'max_value': 128, 'step': 8, 'sampling': None}
activation0 (Choice)
{'default': 'sigmoid', 'conditions': [], 'values': ['sigmoid', 'relu', 'tanh', 'elu'], 'ordered': False}
optimizer (Choice)
{'default': 'rmsprop', 'conditions': [], 'values': ['rmsprop', 'adam', 'adagrad', 'adadelta', 'sgd'], 'ordered': False}
units1 (Int)
{'default': None, 'conditions': [], 'min_value': 8, 'max_value': 128, 'step': 8, 'sampling': None}
activation1 (Choice)
{'default': 'sigmoid', 'conditions': [], 'values': ['sigmoid', 'relu', 'tanh', 'elu'], 'ordered': False}
units2 (Int)
{'default': None, 'conditions': [], 'min_value': 8, 'max_value': 128, 'step': 8, 'sampling': None}
activation2 (Choice)
{'default': 'sigmoid', 'conditions': [], 'values': ['sigmoid', 'relu', 'tanh', 'elu'],

In [46]:
tuner.results_summary()

Results summary
Results in Tuner/Final_Tuning
Showing 10 best trials
Trial summary
Hyperparameters:
num_layers: 1
units0: 120
activation0: elu
optimizer: adam
Score: 0.7922077775001526
Trial summary
Hyperparameters:
num_layers: 2
units0: 96
activation0: sigmoid
optimizer: rmsprop
units1: 24
activation1: relu
units2: 24
activation2: relu
units3: 56
activation3: tanh
units4: 128
activation4: relu
units5: 80
activation5: elu
units6: 128
activation6: relu
units7: 40
activation7: elu
units8: 48
activation8: tanh
Score: 0.7662337422370911
Trial summary
Hyperparameters:
num_layers: 9
units0: 120
activation0: relu
optimizer: adagrad
units1: 32
activation1: tanh
units2: 96
activation2: tanh
units3: 72
activation3: relu
units4: 16
activation4: tanh
units5: 120
activation5: relu
units6: 48
activation6: tanh
units7: 104
activation7: relu
units8: 72
activation8: relu
Score: 0.7077922224998474
Trial summary
Hyperparameters:
num_layers: 4
units0: 24
activation0: relu
optimizer: adam
units1: 8
activat